From : https://docs.python.org/3.6/reference/index.html

In [ ]:
import os
os.getpid()

In [ ]:
import inspect
import hybridcuda
cures = hybridcuda.initcuda()
hybridcuda.registerheader("hybpython.cuh", os.getcwd() + os.sep + ".." + os.sep + ".." + os.sep + "hybpython.cuh")
assert cures == 0

In [ ]:
class hybridkernel:
    gridDimX = 1
    blockDimX = 1
    shared = 0
    stream = 0
    def __init__(self, func):
        self.hc = hybridcuda.processfunction(func)
        self.hc = hybridcuda.cudajitcode(self.hc)
        self.hc = hybridcuda.ptxlinkcode(self.hc)
        
    def __call__(self, *args):
        self.hc = hybridcuda.launch(self.hc, self.gridDimX,1,1, self.blockDimX,1,1, self.shared,self.stream, *args)

    def __getitem__(self, args):
        if (type(args) != tuple):
            self.grid = args
            return self
        # args is a tuple...
        if (len(args) > 0):
            self.grid = args[0]
        if (len(args) > 1):
            self.block = args[1]
        if (len(args) > 2):
            self.shared = args[2]
        if (len(args) > 3):
            self.stream = args[3]
        return self

#decorator definition
def hybridfunction(func):
    return hybridkernel(func)

# 6. Expressions
https://docs.python.org/3/reference/expressions.html


## 6.1 Arithmetic conversions


In [ ]:
@hybridfunction
def arithmetic_convert_001(a,b,c):
    c[0] = a + b

In [ ]:
a = 1
b = 1j
c = [0]

arithmetic_convert_001[1,1](a,b,c)

assert c[0] == (1+1j)

In [ ]:
a = 1
b = 1.0
c = [0]

arithmetic_convert_001[1,1](a,b,c)

assert c[0] == 2.0

In [ ]:
a = 1.0
b = 1.0+2.0j
c = [0]

arithmetic_convert_001[1,1](a,b,c)

assert c[0] == (2.0+2.0j)

In [ ]:
a = 7 / 2
a

## 6.5 Power operator

https://docs.python.org/3/reference/expressions.html#the-power-operator

In [ ]:
@hybridfunction
def arithmetic_power(a,b,c):
    c[0] = a ** b

In [ ]:
c = [1]

arithmetic_power[1,1](2,3,c)
assert c[0] == 8
assert type(c[0]) == int

In [ ]:
c = [1]

arithmetic_power[1,1](2.0,3,c)
assert c[0] == 8.0
assert type(c[0]) == float

In [ ]:
c = [1]

arithmetic_power[1,1](2,3.0,c)
assert c[0] == 8.0
assert type(c[0]) == float

In [ ]:
c = [1]

arithmetic_power[1,1](2.0,3.0,c)
assert c[0] == 8
assert type(c[0]) == float

In [ ]:
c = [1]

arithmetic_power[1,1](1+1j,2,c)
assert abs(c[0]-((1+1j)**2)) < 1e-15
assert type(c[0]) == complex

## 6.6 Unary arithmetic and bitwise operations

https://docs.python.org/3/reference/expressions.html#unary-arithmetic-and-bitwise-operations

In [ ]:
@hybridfunction
def arithmetic_unary_minus(a,b):
    b[0] = -a

In [ ]:
b = [0]

arithmetic_unary_minus[1,1](1,b)
assert b[0] == -1

In [ ]:
@hybridfunction
def arithmetic_unary_plus(a,b):
    b[0] = +a

In [ ]:
b = [0]

arithmetic_unary_plus[1,1](1,b)
assert b[0] == 1

In [ ]:
@hybridfunction
def arithmetic_unary_invert(a,b):
    b[0] = ~a

In [ ]:
b = [0]

arithmetic_unary_invert[1,1](123,b)
assert (b[0] == ~123)

TODO : support operator not, which returns boolean

`a = not 12`

## 6.7 Binary arithmetic operations

https://docs.python.org/3/reference/expressions.html#binary-arithmetic-operations

Mul

In [ ]:
@hybridfunction
def arithmetic_binary_mul(a,b,c):
    c[0] = a * b

In [ ]:
res = [0]

arithmetic_binary_mul[1,1](2,3,res)
assert res[0] == 6
assert type(res[0]) == int

In [ ]:
res = [0]

arithmetic_binary_mul[1,1](2.0,3,res)
assert res[0] == 6.0
assert type(res[0]) == float

In [ ]:
res = [0]

arithmetic_binary_mul[1,1](2.0+1j,3,res)
assert res[0] == 6.0+3.0j
assert type(res[0]) == complex

Add

In [ ]:
@hybridfunction
def arithmetic_binary_add(a,b,c):
    c[0] = a + b

In [ ]:
res = [0]

arithmetic_binary_add[1,1](2,3,res)
assert res[0] == 5
assert type(res[0]) == int

In [ ]:
res = [0]

arithmetic_binary_add[1,1](2.0,3,res)
assert res[0] == 5
assert type(res[0]) == float

In [ ]:
res = [0]

arithmetic_binary_add[1,1](2.0+1j,3,res)
assert res[0] == 5.0+1j
assert type(res[0]) == complex

Sub

In [ ]:
@hybridfunction
def arithmetic_binary_sub(a,b,c):
    c[0] = a - b

In [ ]:
res = [0]

arithmetic_binary_sub[1,1](2,3,res)
assert res[0] == -1
assert type(res[0]) == int

In [ ]:
res = [0]

arithmetic_binary_sub[1,1](2.0,3,res)
assert res[0] == -1.0
assert type(res[0]) == float

In [ ]:
res = [0]

arithmetic_binary_sub[1,1](2.0+1j,3,res)
assert res[0] == -1.0+1j
assert type(res[0]) == complex

In [ ]:
res

Div

In [ ]:
@hybridfunction
def arithmetic_binary_div(a,b,c):
    c[0] = a / b

In [ ]:
res = [0]

arithmetic_binary_div[1,1](6,3,res)
assert res[0] == (6/3)
assert type(res[0]) == type(6/3)

In [ ]:
res = [0]

arithmetic_binary_div[1,1](5,2,res)
assert res[0] == (5/2)
assert type(res[0]) == type(5/2)

FloorDiv

In [ ]:
@hybridfunction
def arithmetic_binary_floordiv(a,b,c):
    c[0] = a // b

In [ ]:
res = [0]

arithmetic_binary_floordiv[1,1](5,2,res)
assert res[0] == (5//2)
assert type(res[0]) == type(5//2)

In [ ]:
res = [0]

arithmetic_binary_floordiv[1,1](5.0,2,res)
assert res[0] == (5.0//2)
assert type(res[0]) == type(5.0//2)

In [ ]:
res = [0]

arithmetic_binary_floordiv[1,1](5,2.0,res)
assert res[0] == (5//2.0)
assert type(res[0]) == type(5//2.0)

Mod

In [ ]:
@hybridfunction
def arithmetic_binary_mod(a,b,c):
    c[0] = a % b

In [ ]:
res = [0]

arithmetic_binary_mod[1,1](5,2,res)
assert res[0] == (5%2)
assert type(res[0]) == type(5%2)

In [ ]:
res = [0]

arithmetic_binary_mod[1,1](5.1,2.3,res)
assert res[0] == (5.1%2.3)
assert type(res[0]) == type(5.1%2.3)

## 6.8 Shifting operations

https://docs.python.org/3/reference/expressions.html#shifting-operations

In [ ]:
@hybridfunction
def arithmetic_shift(a,b,c):
    c[0] = a << b
    c[1] = a >> b

In [ ]:
res = [1,1]

arithmetic_shift[1,1](12,2,res)

assert res[0] == (12 << 2)
assert res[1] == (12 >> 2)
assert type(res[0]) == type(12 << 2)
assert type(res[1]) == type(12 >> 2)

## 6.9 Binary bitwise operations

https://docs.python.org/3/reference/expressions.html#binary-bitwise-operations

In [ ]:
@hybridfunction
def arithmetic_binary_bitwise(a,b,c):
    c[0] = a & b
    c[1] = a ^ b
    c[2] = a | b

In [ ]:
res = [0,0,0]

arithmetic_binary_bitwise(42,167,res)

assert res[0] == (42 & 167)
assert res[1] == (42 ^ 167)
assert res[2] == (42 | 167)
assert type(res[0]) == type(42 & 167)
assert type(res[1]) == type(42 ^ 167)
assert type(res[2]) == type(42 | 167)

In [ ]:
c = 1 == 2
c

In [ ]:
type(c)

## 6.10 Comparisons

https://docs.python.org/3/reference/expressions.html#comparisons

### 6.10.1 Value Comparisons

In [ ]:
@hybridfunction
def compare_values(a,b,c):
    c[0] = a < b
    c[1] = a > b
    c[2] = a == b
    c[3] = a >= b
    c[4] = a <= b
    c[5] = a != b    

In [ ]:
c = [0,0,0,0,0,0]

compare_values[1,1](12.3,15,c)
assert c == [True,False,False,False,True,True]

compare_values[1,1](12,12.3,c)
assert c == [True,False,False,False,True,True]

TODO : strings and other types

## 6.11 Boolean operations

https://docs.python.org/3.6/reference/expressions.html#boolean-operations

In [ ]:
@hybridfunction
def compare_bool_values(a,b,c):
    c[0] = a and b
    c[1] = a or b
    c[2] = not b

In [ ]:
c = [0,0,0]

compare_bool_values[1,1](True, False, c)
assert c == [False, True, True]
compare_bool_values[1,1](False, 1.0, c)
assert c == [1.0 and False, 1.0 or False, not 1.0]


## 6.12 Conditional expressions

https://docs.python.org/3.6/reference/expressions.html#conditional-expressions

In [ ]:
@hybridfunction
def conditional_expression(a,b,c,d):
    d[0] = a if b else c

In [ ]:
d = [1]

conditional_expression[1,1](1.0,False,12,d)
assert d[0] == 12
conditional_expression[1,1](1.0,True,12,d)
assert d[0] == 1.0